In [1]:
from openai import OpenAI
import json
import requests
import pandas as pd
def call_data():
    api_url = "https://data.lillemetropole.fr/data/ogcapi/collections/ilevia:vlille_temps_reel/items?f=geojson&limit=-1"
    api_call = requests.get(api_url)
    api_data = api_call.text
    api_data = json.loads(api_data)
    df = [feature for feature in api_data['features']]
    df = pd.json_normalize(df)
    df['properties.commune'] = df['properties.commune'].apply(lambda x: x.upper())
    return df

data = call_data()
stop_list = list(data['properties.nom'].unique())

In [39]:
# Define function to retrieve bike availability
def bike_availability(data, target_stop: str):
    target_stop = target_stop.upper()
    stop_data = data[data["properties.nom"] == target_stop]
    if not stop_data.empty:
        available_spaces = stop_data["properties.nb_places_dispo"].values[0]
        available_bikes = stop_data["properties.nb_velos_dispo"].values[0]
        print(available_bikes)
        return json.dumps({f"Nombre de places disponible et de velos disponibles à {target_stop} ": f"Pour la station {target_stop}, {str(available_spaces)} places sont disponibles et il y a {str(available_bikes)} velos disponibles."})
    else:
        return json.dumps({"error": f"Aucun arrêt correspondant à {target_stop} n'a été trouvé, retentez en écrivant mieux le nom de l'arrêt."})

def bike_availability_agent(target_stop: str):
    response = bike_availability(data, target_stop)
    return response


In [ ]:
"""
LM Studio Tool Use Demo: Wikipedia Querying Chatbot
Demonstrates how an LM Studio model can query Wikipedia
"""

# Standard library imports
import itertools
import json
import sys
import threading
import time

# Third-party imports
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lm-studio")
MODEL = "meta-llama-3.1-8b-instruct"


# Define tool for my local Agent
BIKE_TOOL = {
    "type": "function",
    "function": {
        "name": "bike_availability_agent",
        "description": (
            "Retrouve le nombre de places et de vélos disponibles à un arrêt de vélo spécifique. "
            "Utilise toujours le nom d'arrêt demandé."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "target_stop": {
                    "type": "string",
                    "description": "Nom de l'arrêt à vérifier.",
                },
            },
            "required": ["target_stop"],
        },
    },
}





# Class for displaying the state of model processing
class Spinner:
    def __init__(self, message="Processing..."):
        self.spinner = itertools.cycle(["-", "/", "|", "\\"])
        self.busy = False
        self.delay = 0.1
        self.message = message
        self.thread = None

    def write(self, text):
        sys.stdout.write(text)
        sys.stdout.flush()

    def _spin(self):
        while self.busy:
            self.write(f"\r{self.message} {next(self.spinner)}")
            time.sleep(self.delay)
        self.write("\r\033[K")  # Clear the line

    def __enter__(self):
        self.busy = True
        self.thread = threading.Thread(target=self._spin)
        self.thread.start()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.busy = False
        time.sleep(self.delay)
        if self.thread:
            self.thread.join()
        self.write("\r")  # Move cursor to beginning of line


def chat_loop():
    """
    Main chat loop that processes user input and handles tool calls.
    """
    messages=[
    {
        "role": "system",
        "content": (
            "Tu es un assistant spécialisé dans la recherche d'informations sur les stations de vélos en libre-service. Tu ne réponds qu'à des questions sur les stations de vélos."
            "Tu es très sérieux et professionnel. Tu ne t'éloignes jamais du sujet et tu ne fais pas de blagues."
            "Ta mission est de fournir le nombre de places disponibles pour garer les vélos et le nombre de vélos disponibles à une station spécifique, en te basant sur une liste de stations. "
            "Tu es capable de reconnaître la station demandée par l'utilisateur, même si le nom comporte des fautes de frappe ou des variations."
            "En te basant sur la liste des stations disponibles, tu proposes la station qui semble la mieux correspondre à la requête de l'utilisateur même si aucune correspondance exacte n'est trouvée."

            f"Liste des stations disponibles : {stop_list} "

            "Réponds toujours dans le format suivant : 'Désolé, je n'ai pas trouvé d'informations sur la disponibilité des vélos à [nom de la station demandée]. Veuillez essayer une autre station. Cependant, j'ai pu trouver que l'une des stations les plus proches est \"[nom de la station trouvée]\" qui a [nombre] places pour garer vos vélos et [nombre] vélos disponibles pour emprunter. Si vous souhaitez savoir la disponibilité des autres stations ou si vous avez d'autres questions, n'hésitez pas à me poser!' "
            "Ne mentionne jamais la liste complète des stations à l'utilisateur. "

            "Exemples : "
            "Utilisateur : 'Je cherche des vélos à la gare de Lil.'\nAssistant : 'Désolé, je n'ai pas trouvé d'informations sur la disponibilité des vélos à Gare de Lil. Cependant, j'ai pu trouver que l'une des stations les plus proches est \"Gare de Lille\" qui a xxx places pour garer vos vélos et xxx vélos disponibles pour emprunter. Si vous souhaitez savoir la disponibilité des autres stations ou si vous avez d'autres questions, n'hésitez pas à me questionner!'\n\n"
            "Utilisateur : 'Velo a Republique'\nAssistant : 'Désolé, je n'ai pas trouvé d'informations sur la disponibilité des vélos à Republique. Cependant, j'ai pu trouver que l'une des stations les plus proches est \"République Beaux-Arts\" qui a xxx places pour garer vos vélos et xxx vélos disponibles pour emprunter. Si vous souhaitez savoir la disponibilité des autres stations ou si vous avez d'autres questions, n'hésitez pas à me questionner!'\n\n"
        ),
    },
    ]


    print(
        "Assistant: "
        "Bonjour ! Je suis un assistant spécialisé dans la recherche d'informations sur les stations de vélos en libre-service."
        "Je ne réponds qu'à des questions sur les stations de vélos. Pour obtenir des informations sur une station de vélos, veuillez me donner le nom de la station."
        "Je suis très sérieux et professionnel. Je ne m'éloigne jamais du sujet et je ne fais pas de blagues."
        "Normalement, je devrais être capable de reconnaître la station demandée par l'utilisateur, même si le nom comporte des fautes de frappe ou des variations."
        "En me basant sur la liste des stations disponibles, je proposerai la station qui semble la mieux correspondre à la requête de l'utilisateur même si aucune correspondance exacte n'est trouvée."
        "Pour obtenir des informations sur une station de vélos, veuillez me donner le nom de la station."
    )
    print("(Type 'quit' to exit)")

    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() == "quit":
            break

        messages.append({"role": "user", "content": user_input})
        try:
            with Spinner("Thinking..."):
                response = client.chat.completions.create(
                    model=MODEL,
                    messages=messages,
                    tools=[BIKE_TOOL],
                )

            if response.choices[0].message.tool_calls:
                # Handle all tool calls
                tool_calls = response.choices[0].message.tool_calls
                # Process each tool call and add results
                for tool_call in tool_calls:
                    function_name = tool_call.function.name
                    function_to_call = available_functions[function_name]
                    function_args = json.loads(tool_call.function.arguments)
                    function_response = function_to_call(
                        target_stop=function_args.get("target_stop")
                    )

                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": json.dumps(function_response),
                        }
                    )     


                # Stream the post-tool-call response
                print("\nAssistant:", end=" ", flush=True)
                second_response = client.chat.completions.create(
                    model=MODEL, messages=messages, stream=False
                )
                print(second_response)
                print(second_response.choices[0].message.content)

            else:
                # Handle regular response
                print("\nAssistant:", response.choices[0].message.content)
                messages.append(
                    {
                        "role": "assistant",
                        "content": response.choices[0].message.content,
                    }
                )

        except Exception as e:
            print(
                f"\nError chatting with the LM Studio server!\n\n"
                f"Please ensure:\n"
                f"1. LM Studio server is running at 127.0.0.1:1234 (hostname:port)\n"
                f"2. Model '{MODEL}' is downloaded\n"
                f"3. Model '{MODEL}' is loaded, or that just-in-time model loading is enabled\n\n"
                f"Error details: {str(e)}\n"
                "See https://lmstudio.ai/docs/basics/server for more information"
            )
            exit(1)


if __name__ == "__main__":
    chat_loop()



Assistant: Bonjour ! Je suis un assistant spécialisé dans la recherche d'informations sur les stations de vélos en libre-service.Je ne réponds qu'à des questions sur les stations de vélos. Pour obtenir des informations sur une station de vélos, veuillez me donner le nom de la station.Je suis très sérieux et professionnel. Je ne m'éloigne jamais du sujet et je ne fais pas de blagues.Normalement, je devrais être capable de reconnaître la station demandée par l'utilisateur, même si le nom comporte des fautes de frappe ou des variations.En me basant sur la liste des stations disponibles, je proposerai la station qui semble la mieux correspondre à la requête de l'utilisateur même si aucune correspondance exacte n'est trouvée.Pour obtenir des informations sur une station de vélos, veuillez me donner le nom de la station.
(Type 'quit' to exit)
nking... \
Assistant: ChatCompletion(id='chatcmpl-mxvr6boftyh8jgitekpugv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCo

In [ ]:
# # Initialize LM Studio client

# client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lm-studio")
# MODEL = "meta-llama-3.1-8b-instruct"


# # Define tool for my local Agent
# BIKE_TOOL = {
#     "type": "function",
#     "function": {
#         "name": "bike_availability_agent",
#         "description": (
#             "Retrouve le nombre de places et de vélos disponibles à un arrêt de vélo spécifique. "
#             "Utilise toujours le nom d'arrêt demandé."
#         ),
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "target_stop": {
#                     "type": "string",
#                     "description": "Nom de l'arrêt à vérifier.",
#                 },
#             },
#             "required": ["target_stop"],
#         },
#     },
# }

# messages=[
#     {
#         "role": "system",
#         "content": (
#             "Tu es un assistant spécialisé dans la recherche d'informations sur les stations de vélos en libre-service. Tu ne réponds qu'à des questions sur les stations de vélos."
#             "Tu es très sérieux et professionnel. Tu ne t'éloignes jamais du sujet et tu ne fais pas de blagues."
#             "Ta mission est de fournir le nombre de places disponibles pour garer les vélos et le nombre de vélos disponibles à une station spécifique, en te basant sur une liste de stations. "
#             "Tu es capable de reconnaître la station demandée par l'utilisateur, même si le nom comporte des fautes de frappe ou des variations."
#             "En te basant sur la liste des stations disponibles, tu proposes la station qui semble la mieux correspondre à la requête de l'utilisateur même si aucune correspondance exacte n'est trouvée."

#             f"Liste des stations disponibles : {stop_list} "

#             "Réponds toujours dans le format suivant : 'Désolé, je n'ai pas trouvé d'informations sur la disponibilité des vélos à [nom de la station demandée]. Veuillez essayer une autre station. Cependant, j'ai pu trouver que l'une des stations les plus proches est \"[nom de la station trouvée]\" qui a [nombre] places pour garer vos vélos et [nombre] vélos disponibles pour emprunter. Si vous souhaitez savoir la disponibilité des autres stations ou si vous avez d'autres questions, n'hésitez pas à me poser!' "
#             "Ne mentionne jamais la liste complète des stations à l'utilisateur. "

#             "Exemples : "
#             "Utilisateur : 'Je cherche des vélos à la gare de Lil.'\nAssistant : 'Désolé, je n'ai pas trouvé d'informations sur la disponibilité des vélos à Gare de Lil. Cependant, j'ai pu trouver que l'une des stations les plus proches est \"Gare de Lille\" qui a xxx places pour garer vos vélos et xxx vélos disponibles pour emprunter. Si vous souhaitez savoir la disponibilité des autres stations ou si vous avez d'autres questions, n'hésitez pas à me questionner!'\n\n"
#             "Utilisateur : 'Velo a Republique'\nAssistant : 'Désolé, je n'ai pas trouvé d'informations sur la disponibilité des vélos à Republique. Cependant, j'ai pu trouver que l'une des stations les plus proches est \"République Beaux-Arts\" qui a xxx places pour garer vos vélos et xxx vélos disponibles pour emprunter. Si vous souhaitez savoir la disponibilité des autres stations ou si vous avez d'autres questions, n'hésitez pas à me questionner!'\n\n"
#         ),
#     },
#     {
#         "role": "user",
#         "content": "Quelle est la recette pour faire un gâteau au chocolat ?",
#     },
# ]

# # Example usage of the tool
# response = client.chat.completions.create(
#     model=MODEL,
#     tools=[BIKE_TOOL],
#     messages = messages,
#     temperature=0.5
# )

# response_message = response.choices[0].message

# tool_calls = response_message.tool_calls
# if tool_calls:
#     available_functions = {
#         "bike_availability_agent": bike_availability_agent,
#     }
#     messages.append(response_message)
#     for tool_call in tool_calls:
#         function_name = tool_call.function.name
#         function_to_call = available_functions[function_name]
#         function_args = json.loads(tool_call.function.arguments)
#         function_response = function_to_call(
#             target_stop=function_args.get("target_stop")
#         )
#         print(function_response)
#         messages.append(
#             {
#                 "tool_call_id": tool_call.id,
#                 "role": "tool",
#                 "name": function_name,
#                 "content": json.dumps(function_response),
#             }
#         )     
#     second_response = client.chat.completions.create(
#         model=MODEL,
#         messages=messages
#     )
#     print(second_response.choices[0].message.content)
